In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 3.9 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 6.5 MB 53.4 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.0 MB/s 
     |████████████████████████████████| 134 kB 70.8 MB/s 
     |████████████████████████████████| 1.1 MB 66.3 MB/s 
     |████████████████████████████████| 212 kB 68.9 MB/s 
     |████████████████████████████████| 127 kB 69.7 MB/s 
     |████████████████████████████████| 271 kB 69.6 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 144 kB 57.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
import numpy as np
import pandas as pd
import re
import random
import torch
from tqdm.notebook import tqdm
import transformers
from datasets import load_dataset
import textwrap

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
dataset = load_dataset("huggingartists/oxxxymiron")

Using custom data configuration default
Reusing dataset lyrics_dataset (/root/.cache/huggingface/datasets/huggingartists___lyrics_dataset/default/1.0.0/61ef97b77f14b682ef94b166646e86c71ce5d2a2b81c608078a8dd4355f0f7d6)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [ ]:
dataset = ' '.join(dataset['train']['text'])

In [ ]:
dataset = dataset.replace('\n', ' ')

In [ ]:
dataset

'Говно, залупа, пенис, хер, давалка, хуй, блядина Головка, шлюха, жопа, член, еблан, петух… мудила Рукоблуд, ссанина, очко, блядун, вагина Сука, ебланище, влагалище, пердун, дрочила Пидор, пизда, туз, малафья Гомик, мудила, пилотка, манда Анус, вагина, путана, пидрила Шалава, хуила, мошонка, елда… Раунд! Вот как я должен был баттлить Дуню И тех, кто калякал его стишки Но там перебивали его дружки Пожелайте им рака прямой кишки Я был не на коне, я об этом помню Порой с оппонентом идёшь впритык Но сегодня я здесь, и сегодня я в форме Сегодня мой хуй отсосёшь мне ты Клянусь, я готовился две недели, ты — явно год Но для беспокойства повод не вижу Даже если ты прыгнешь выше головы, как ебаный кот В рассказе Эдгара По, ты все равно на голову ниже И почему все ждали баттл, как битву титанов или херачущих файерболами? Страшные заклинания, летящие во все стороны? Панчи, что мне объявят единогласный импичмент? Какого хуя? Для меня баттлы с Криплом и Дуней были гораздо эпичней Там хоть бой ветера

In [ ]:
tokens = tokenizer.encode(dataset, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))

179818


In [ ]:
tokens.shape

(179818,)

In [ ]:
from transformers import GPT2LMHeadModel, AdamW

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);

In [ ]:
batch_size = 5
max_len = 256
epochs = 16

train = len(tokens)//(batch_size*max_len)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# готовим тензоры для обучения

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    model.train()
    pbar = tqdm(range(train))
    for i in pbar:
        batch_ids = prep_tensors(tokens, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()


epoch 1/16 : training


  0%|          | 0/140 [00:00<?, ?it/s]

epoch 2/16 : training


  0%|          | 0/140 [00:00<?, ?it/s]

epoch 3/16 : training


...

  0%|          | 0/140 [00:00<?, ?it/s]

epoch 15/16 : training


  0%|          | 0/140 [00:00<?, ?it/s]

epoch 16/16 : training


  0%|          | 0/140 [00:00<?, ?it/s]

In [ ]:
prompt = 'Эй'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=69,
    num_beams=3,
    do_sample=True,
    temperature=1.,
    top_k=5,
    top_p=0.5,
    no_repeat_ngram_size=1,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Эй, я не знаю как тебя зовут Но если ты хочешь знать моё имя — то называй меня Мирон! И покажи мне свой зад Я покажу
тебе своё и скажу: «Хватит пиздеть» Ты скажешь что-то ещё? Да бросьте вы нахуй ваши треки Вы же сами говорили про это в
прошлый раз.
------------------
Эй, ты не прав! Ты просто блефуешь и втирал про себя Но я тебя вижу насквозь Я знаю всё это из-за того что у меня нет ни
гроша А если бы был — то мы были б на месте ваших оппонентов Ведь вы сами говорили мне: «Не подходи к ней» И знаете
почему? Потому
------------------
Эй Эй, эй! Я не знаю как у вас ноунеймы Но я слышал что-то вроде: «Скажи мне кто твой друг» И ты спросишь — а зачем?
Ведь он может и так сказать Что это за хуйня в твоей фразе Ты же сам знаешь ответ на свой вопрос. Это пиздец какой то
Если
------------------
Эйф — это не тот, что в фильме «Тёмная сторона Луны» Ты же знаешь: я всё перевоспитываю и возвращаю к истокам Но мне
скучно жить с этим говном И если бы у меня были деньги на новый альбом Я б продал ег

In [ ]:
torch.save(model, '../model_weights//oxxxymiron.pth')